In [1]:
import torch

# Import models
from net.densenet import densenet121
from net.resnet import resnet50, resnet110
from net.vgg import vgg16
from net.wide_resnet import wrn_28_10
from net.inception import inceptionv3

In [2]:
# Test a single model or a single ensemble

ensemble = []

with torch.no_grad():
    for i in range(5):
        model = densenet121().cuda()
        model.load_state_dict(torch.load(f'../ood_ensemble/densenet121/densenet121_{(i+1)}_350.model'))
        ensemble.append(model)

In [3]:
# Load the data loaders
import data.cifar10_100 as cifar10_100
import data.svhn as svhn
import data.cifar10_ood as cifar10_ood
from data.cifar10_ood import CIFAR10_OOD

cifar10_test_loader = cifar10_100.get_loaders(128,
                                              dataset='cifar10',
                                              train=False)
cifar100_test_loader = cifar10_100.get_loaders(128,
                                               dataset='cifar100',
                                               train=False)

svhn_test_loader = svhn.get_loaders(128,
                                    dataset='svhn',
                                    train=False)

cifar10_ood = CIFAR10_OOD(path='./data/ood_generated')
cifar10_ood_loader = torch.utils.data.DataLoader(
                         cifar10_ood,
                         batch_size=128,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True,
                     )

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./datasets/test_32x32.mat


In [4]:
from metrics.uncertainty_confidence import entropy, confidence
from metrics.ood_metrics import get_roc_auc
from metrics.ood_metrics import get_roc_auc_ensemble

svhn_entropy_aurocs = []
# svhn_confidence_aurocs = []

cifar100_entropy_aurocs = []
#cifar100_confidence_aurocs = []
 
cifar10_ood_entropy_aurocs = []
# cifar10_ood_confidence_aurocs = []

with torch.no_grad():
    for i, model in enumerate(ensemble):
        print (f'Model {i}')
        (_, _, _), (_, _, _), svhn_entropy_auroc, _ = get_roc_auc_ensemble(ensemble,
                                                                  cifar10_test_loader,
                                                                  svhn_test_loader,
                                                                  uncertainty='predictive_entropy',
                                                                  device="cuda"
                                                      )
#         (_, _, _), (_, _, _), svhn_confidence_auroc, _ = get_roc_auc(model,
#                                                                   cifar10_test_loader,
#                                                                   svhn_test_loader,
#                                                                   uncertainty=confidence,
#                                                                   device="cuda",
#                                                                   confidence=True
#                                                       )
        print ('SVHN done')
        (_, _, _), (_, _, _), cifar100_entropy_auroc, _ = get_roc_auc_ensemble(ensemble,
                                                                  cifar10_test_loader,
                                                                  cifar100_test_loader,
                                                                  uncertainty='predictive_entropy',
                                                                  device="cuda"
                                                      )
#         (_, _, _), (_, _, _), cifar100_confidence_auroc, _ = get_roc_auc(model,
#                                                                   cifar10_test_loader,
#                                                                   cifar100_test_loader,
#                                                                   uncertainty=confidence,
#                                                                   device="cuda",
#                                                                   confidence=True
#                                                       )
        print ('CIFAR100 done')
        (_, _, _), (_, _, _), cifar10_ood_entropy_auroc, _ = get_roc_auc_ensemble(ensemble,
                                                                  cifar10_test_loader,
                                                                  cifar10_ood_loader,
                                                                  uncertainty='predictive_entropy',
                                                                  device="cuda"
                                                      )
#         (_, _, _), (_, _, _), cifar10_ood_confidence_auroc, _ = get_roc_auc(model,
#                                                                   cifar10_test_loader,
#                                                                   cifar10_ood_loader,
#                                                                   uncertainty=confidence,
#                                                                   device="cuda",
#                                                                   confidence=True
#                                                       )
        print ('CIFAR10 OoD done')

        svhn_entropy_aurocs.append(svhn_entropy_auroc)
        #svhn_confidence_aurocs.append(svhn_confidence_auroc)
 
        cifar100_entropy_aurocs.append(cifar100_entropy_auroc)
        #cifar100_confidence_aurocs.append(cifar100_confidence_auroc)

        cifar10_ood_entropy_aurocs.append(cifar10_ood_entropy_auroc)
        #cifar10_ood_confidence_aurocs.append(cifar10_ood_confidence_auroc)

Model 0
SVHN done
CIFAR100 done
CIFAR10 OoD done
Model 1
SVHN done
CIFAR100 done
CIFAR10 OoD done
Model 2
SVHN done
CIFAR100 done
CIFAR10 OoD done
Model 3
SVHN done
CIFAR100 done
CIFAR10 OoD done
Model 4
SVHN done
CIFAR100 done
CIFAR10 OoD done


In [5]:
svhn_entropy_aurocs = torch.tensor(svhn_entropy_aurocs)
#svhn_confidence_aurocs = torch.tensor(svhn_confidence_aurocs)

cifar100_entropy_aurocs = torch.tensor(cifar100_entropy_aurocs)
#cifar100_confidence_aurocs = torch.tensor(cifar100_confidence_aurocs)

cifar10_ood_entropy_aurocs = torch.tensor(cifar10_ood_entropy_aurocs)
#cifar10_ood_confidence_aurocs = torch.tensor(cifar10_ood_confidence_aurocs)

In [6]:
print ('SVHN: ===================>')
print (f'Entropy AUROC: {torch.mean(svhn_entropy_aurocs)} +- {torch.std(svhn_entropy_aurocs)}')
#print (f'Confidence AUROC: {torch.mean(svhn_confidence_aurocs)} +- {torch.std(svhn_confidence_aurocs)}')

print ('CIFAR-100: ===================>')
print (f'Entropy AUROC: {torch.mean(cifar100_entropy_aurocs)} +- {torch.std(cifar100_entropy_aurocs)}')
#print (f'Confidence AUROC: {torch.mean(cifar100_confidence_aurocs)} +- {torch.std(cifar100_confidence_aurocs)}')

print ('Morphed CIFAR-10: ===================>')
print (f'Entropy AUROC: {torch.mean(cifar10_ood_entropy_aurocs)} +- {torch.std(cifar10_ood_entropy_aurocs)}')
#print (f'Confidence AUROC: {torch.mean(cifar10_ood_confidence_aurocs)} +- {torch.std(cifar10_ood_confidence_aurocs)}')


SVHN: ===================>
Entropy AUROC: 0.9751892386293793 +- 2.1040356283291935e-09
CIFAR-100: ===================>
Entropy AUROC: 0.9142333350000001 +- 1.2412670766236366e-16
Morphed CIFAR-10: ===================>
Entropy AUROC: 0.8263104626761265 +- 1.2412670766236366e-16
